# **I.IMPORT THƯ VIỆN**

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

!pip install googletrans==4.0.0-rc1 # Installing the library
from googletrans import Translator

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=79c99adf564e76d0e5940e854c69a7be203219a37120fd04dddab8d95c39382e
  Stored in directory: /root/.cache/pip/wheels/95/0f/04/b17a72024b56a60e499ce1a6313d283ed5ba332407155bee03
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

# **II. XÂY DỰNG CLASS TIỀN XỬ LÝ**

**HƯỚNG TIẾP CẬN**

1. Một **class DataPreprocessor** chung
2. Một **class Pipeline** xử lý theo từng loại file
3. Các hàm áp dụng tự động cho cả 4 file


In [ ]:
class DataPreprocessor:
      """
    Class DataPreprocessor cung cấp các phương thức để xử lý và kiểm tra dữ liệu
    trước khi huấn luyện mô hình Machine Learning.

    Chức năng chính:
    1. Đọc dữ liệu từ file CSV.
    2. Kiểm tra thông tin tổng quan của dataset.
    3. Xử lý giá trị missing theo nhiều chiến lược.
    4. Loại bỏ dòng trùng lặp.
    5. Xuất dữ liệu đã xử lý ra file CSV.

    Attributes
    ----------
    df : pandas.DataFrame
        DataFrame lưu trữ dữ liệu hiện tại đang được xử lý.
      """
    def __init__(self):
        """
        Khởi tạo object DataPreprocessor.
        """
       self.df = None

    # =========================
    # 1. Đọc dữ liệu
    # =========================
    def load(self, filepath):
        """
        Đọc dữ liệu từ file CSV và lưu vào thuộc tính df.

        Parameters
        ----------
        filepath : str
            Đường dẫn đến file CSV cần đọc.

        Returns
        -------
        pandas.DataFrame
            DataFrame chứa dữ liệu đã đọc.
        """
        self.df = pd.read_csv(filepath)
        print("Đã đọc dữ liệu thành công!")
        return self.df

    # =========================
    # 2. Kiểm tra thông tin dữ liệu
    # =========================
    def inspect(self):
        """
        Kiểm tra và hiển thị thông tin tổng quan về dataset hiện tại.

        In ra:
        - Thông tin chi tiết về các cột và kiểu dữ liệu.
        - Kích thước dataset (số dòng, số cột).
        - 5 dòng dữ liệu đầu tiên.
        - Thống kê mô tả các cột số.
        - Tỷ lệ giá trị NULL.
        - Tỷ lệ dòng trùng lặp.
        """

        print("Thông tin dữ liệu:")
        print(self.df.info())

        print("\nSố dòng và cột:")
        print(self.df.shape)

        print("\n 5 dòng đầu tiên:")
        print(self.df.head())

        print("\nThống kê mô tả:")
        print(self.df.describe())

        print("\n Tỷ lệ NULL:")
        print(self.df.isnull().mean())

        print("\n Tỷ lệ duplicate:")
        print(self.df.duplicated().mean())


    # =========================
    # 3. Xử lý missing values theo cols + strategy
    # =========================
    def handle_missing(self, cols=None, strategy="drop"):
        """
        cols: list tên cột cần xử lý
        strategy: "mean", "median", "mode", "unknown", "drop"
        """

        if cols is None:
            print("Không truyền cột nào → không xử lý missing value.")
            return self.df

        for col in cols:

            if col not in self.df.columns:
                print(f"Cột '{col}' không tồn tại → bỏ qua.")
                continue

            if self.df[col].isnull().sum() == 0:
                continue  # Không có missing → bỏ

            # === CHIẾN LƯỢC ÁP DỤNG ===
            if strategy == "mean":
                self.df[col] = self.df[col].fillna(self.df[col].mean())

            elif strategy == "median":
                self.df[col] = self.df[col].fillna(self.df[col].median())

            elif strategy == "mode":
                self.df[col] = self.df[col].fillna(self.df[col].mode()[0])

            elif strategy == "unknown":
                self.df[col] = self.df[col].fillna("Unknown")

            elif strategy == "drop":
                self.df = self.df[self.df[col].notnull()]

            else:
                print(f"Strategy '{strategy}' không hợp lệ!")

        print(" Đã xử lý missing theo strategy!")
        return self.df

    # =========================
    # 4. Xử lý duplicate
    # =========================
    def remove_duplicates(self):
        """
        Loại bỏ các dòng dữ liệu trùng lặp trong DataFrame.

        Returns
        -------
        pandas.DataFrame
            DataFrame đã loại bỏ các dòng trùng lặp.
        """
        before = self.df.shape[0]
        duplicated_rows = self.df.duplicated().sum()   # số dòng trùng

        if duplicated_rows == 0:
            print("Không có dòng trùng lặp nào trong dữ liệu.")
            return self.df

        # Nếu có duplicate → xoá
        self.df = self.df.drop_duplicates()
        after = self.df.shape[0]

        print(f"Đã xóa {before - after} dòng trùng lặp!")
        return self.df

    # -------------------------
    # 5. EXPORT
    # -------------------------
    def export(self, path):
        """
        Xuất DataFrame hiện tại ra file CSV.

        Parameters
        ----------
        path : str
            Đường dẫn và tên file CSV sẽ lưu.
        """
        self.df.to_csv(path, index=False)



In [ ]:
class DatasetPipeline:
      """
    Class DatasetPipeline thực hiện quy trình tiền xử lý dữ liệu (ETL) cho các bảng dataset khác nhau
    dựa trên DataPreprocessor.

    Parameters
    ----------
    preprocessor : DataPreprocessor
        Object của lớp DataPreprocessor để thực hiện load, xử lý missing, duplicate, và export dữ liệu.

    Methods
    -------
    process_user(path_in, path_out)
        Xử lý bảng user: missing, duplicate, chuẩn hóa cột income, living_with, nation, job.

    process_context(path_in, path_out)
        Xử lý bảng context: missing, duplicate, chuẩn hóa cột go_with, weather, time, viettel_no_*.

    process_mobile_plan_user(path_in, path_out, path_out_agg=None)
        Xử lý bảng mobile_plan_user: missing, duplicate và tạo bảng tổng hợp theo id.

    process_mobile_plan_attr(path_in, path_out)
        Xử lý bảng mobile_plan_attr: missing, duplicate và export dữ liệu đã clean.
        """

    def __init__(self, preprocessor):
        """
        Khởi tạo DatasetPipeline với DataPreprocessor.
        """
        self.p = preprocessor

    # ===========================
    # 1. USER
    # ===========================
    def process_user(self, path_in, path_out):
        """
        Tiền xử lý bảng user và xuất file đã clean.

        Parameters
        ----------
        path_in : str
            File CSV raw.
        path_out : str
            File CSV đã clean.
        """

        # Load file
        df = self.p.load(path_in)

        # In ra thông tin cơ bản
        self.p.inspect()

        # Xử lý missing: Thay giá trị NULL trong cột education bằng "Unknown"
        self.p.handle_missing(cols = ["education"], strategy="unknown")

        # Xử lý duplicate
        self.p.remove_duplicates()

        # Xử lý riêng cho bảng user
        # income
        if 'income' in df.columns:
            # Loại bỏ khoảng trắng thừa và dấu phẩy
            df['income'] = df['income'].str.replace(',', '', regex=False).str.strip()
            # Chuyển từng giá trị thành số VND
            income_list = []
            for val in df['income']:
                val = val.strip()  # bỏ khoảng trắng giữa giá trị số và VND/USD
                if '$' in val:     # nếu là USD
                    number = float(val.replace('$','').strip())
                    number = number * 24000
                    income_list.append(number)
                elif 'VND' in val: # nếu là VND
                    number = float(val.replace('VND','').strip())
                    income_list.append(number)
                else:              # nếu không có đơn vị
                    income_list.append(float(val))
        # Gán lại giá trị cho cột Income
        df['income'] = income_list


        # living_with
        if 'living_with' in df.columns:
            # Thay thế khoảng trắng trong chuỗi bằng rỗng:
            df['living_with'] = df['living_with'].str.replace(' ', '')

            # Tách thành 2 cột marriage_status và number_child để thuận tiện cho phân tích
            df[['marriage_status','number_child']] = df['living_with'].str.split('_', expand=True)
            # Chuyển dữ liệu cột "Số con" thành dạng int
            df['number_child'] = df['number_child'].astype(int)


        # nation: Chuyển thành dạng title
        if 'nation' in df.columns:
            df['nation'] = df['nation'].str.title()

        # job: Đồng nhất hóa các ngôn ngữ sang tiếng anh
        if 'job' in df.columns:
            translator = Translator()
            # Lấy danh sách job duy nhất
            unique_jobs = df['job'].unique()
            # Dịch từng job duy nhất
            translated_dict = {}
            for job in unique_jobs:
                try:
                    translated_text = translator.translate(job, src='auto', dest='en').text
                    translated_dict[job] = translated_text
                except:
                    translated_dict[job] = job  # giữ nguyên nếu lỗi
            # Map kết quả dịch về cột gốc
            df['job'] = df['job'].map(translated_dict)

        # Xuất file
        self.p.export(path_out)
        print("✔ user.csv DONE")


    # 2. context
    # ===========================
    def process_context(self, path_in, path_out):
        """
        Tiền xử lý bảng context và xuất file đã clean.

        Parameters
        ----------
        path_in : str
            File CSV raw.
        path_out : str
            File CSV đã clean.
        """

        # Load data
        df = self.p.load(path_in)

        # In ra thông tin cơ bản
        self.p.inspect()

        # Xử lý missing:
        self.p.handle_missing()

        # Xử lý duplicate
        self.p.remove_duplicates()

        # Xử lý riêng cho bảng context

        # go_with và weather
        # Hàm để làm sạch dữ liệu 2 cột go_with và weather: Loại bỏ các ký tự đặc biệt
        def normalize(text):
            if pd.isna(text):
                return text
            # Bước 1: Dùng Regex thay thế tất cả ký tự KHÔNG phải chữ cái (a-z, A-Z) thành rỗng
            # Điều này sẽ xóa cả dấu câu, ký tự đặc biệt và khoảng trắng thừa
            clean_text = re.sub(r'[^a-zA-Z]', '', str(text))

            # Bước 2: Chuẩn hóa viết hoa chữ đầu (Title Case)
            # Kết quả lúc này sẽ là: (Alone, Family, Friends) / (Sunny,Snowy, Rainy)
            clean_text = clean_text.capitalize()
            return clean_text
        # Áp dụng cho các cột nếu tồn tại
        for col in ['go_with', 'weather']:
            if col in df.columns:
                df[col] = df[col].apply(normalize)


        # time
        def convert_time(t):
            if pd.isna(t):
                return t
            t = str(t).strip()

            # Thêm khoảng trắng nếu dạng '7PM' → '7 PM'
            t = re.sub(r'(\d)([APap][Mm])$', r'\1 \2', t)

            # Thử parse nhiều định dạng phổ biến
            for fmt in ['%I %p', '%I:%M %p', '%H:%M', '%H']:
                try:
                      return pd.to_datetime(t, format=fmt).strftime('%H:00')
                except:
                      continue

            # Nếu không parse được → giữ nguyên
            return t
        # Áp dụng hàm convert_time cho cột time
        if 'time' in df.columns:
            df['time'] = df['time'].apply(convert_time)


        # viettel_no_0, viettel_no_1, viettel_no_2
        def fix_viettel(row):
            # Nếu 45 phút (viettel_no_2 = 1) → hai cột còn lại = 0
            if row['viettel_no_2'] == 1:
                return 0, 0, 1
            # Nếu 30 phút (viettel_no_1 = 1) → hai cột còn lại = 0
            elif row['viettel_no_1'] == 1:
                return 0, 1, 0
            # Nếu 15 phút (viettel_no_0 = 1) → hai cột còn lại = 0
            elif row['viettel_no_0'] == 1:
                return 1, 0, 0
            # Nếu cả 3 đều 0 → giữ nguyên 0-0-0
            else:
                return 0, 0, 0
        # Áp dụng cho 3 cột viettel_no_0, 1, 2
        df[['viettel_no_0','viettel_no_1','viettel_no_2']] = df.apply(fix_viettel, axis=1, result_type='expand')


        # Xuất file
        self.p.export(path_out)
        print("✔ context.csv DONE")



    # 3. mobile_plan_user
    # ===========================
    def process_mobile_plan_user(self, path_in, path_out, path_out_agg=None):
        """
        Tiền xử lý mobile_plan_user, tạo bảng tổng hợp theo ID (tùy chọn) và xuất file.
        path_in: đường dẫn file raw
        path_out: đường dẫn file cleaned
        path_out_agg: (tùy chọn) đường dẫn để lưu bảng tổng hợp theo ID
        """
        # Load data
        df = self.p.load(path_in)

        # In ra thông tin cơ bản
        self.p.inspect()

        # Xử lý missing: Drop các NULL ở 2 cột mobile_plan, accept vì tỷ lệ Null thấp
        self.p.handle_missing(cols=['mobile_plan', 'accept'], strategy="drop")

        # Xử lý duplicate
        self.p.remove_duplicates()

        # ===========================
        # Tạo bảng tổng hợp theo ID
        # ===========================
        mobile_plan_agg = df.groupby('id').agg(
            number_of_suggestions=('mobile_plan', 'count'),
            number_of_plans=('mobile_plan', 'nunique'),
            number_of_accept=('accept', 'sum'),
            number_of_accept_2=(
                'mobile_plan',
                lambda x: x[df.loc[x.index, 'accept'] == 1].nunique()
            ),
            mobile_plan=(
                'mobile_plan',
                lambda x: ', '.join(
                    x[df.loc[x.index, 'accept'] == 1].dropna().unique()
                ) if x[df.loc[x.index, 'accept'] == 1].nunique() > 0 else 'Unknown'
            )
        ).reset_index()

        # Xuất file cleaned
        self.p.export(path_out)
        print("✔ mobile_plan_user.csv DONE")

        # Xuất file tổng hợp nếu có đường dẫn
        if path_out_agg:
            mobile_plan_agg.to_csv(path_out_agg, index=False)
            print("✔ mobile_plan_agg.csv DONE")


    # 4. mobile_plan_attr
    # ===========================
    def process_mobile_plan_attr(self, path_in, path_out):
        """
        Tiền xử lý mobile_plan_attr và xuất file đã clean.

        Parameters
        ----------
        path_in : str
            File CSV raw.
        path_out : str
            File CSV đã clean.
        """

        # Load data
        df = self.p.load(path_in)

        # In ra thông tin cơ bản
        self.p.inspect()


        # Xử lý missing:
        self.p.handle_missing()


        # Xử lý duplicate
        self.p.remove_duplicates()

        # Xuất file cleaned
        self.p.export(path_out)
        print("✔ mobile_plan_attr.csv DONE")

# **III. ÁP DỤNG**

## **1. Áp dụng cho tập train**

In [ ]:
# Chạy cho user
def run_user():
    p = DataPreprocessor()
    pipe = DatasetPipeline(p)

    pipe.process_user("user.csv", "clean_user.csv")
    print("✔ user.csv processed → clean_user.csv")

run_user()

# Đọc lại file clean_user
clean_user = pd.read_csv("clean_user.csv")
print("\n 5 dòng đầu tiên sau khi làm sạch user.csv:")
clean_user.head()

Đã đọc dữ liệu thành công!
Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11572 entries, 0 to 11571
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 11572 non-null  int64 
 1   name               11572 non-null  object
 2   gender             11572 non-null  object
 3   age                11572 non-null  int64 
 4   education          7569 non-null   object
 5   profession         11572 non-null  object
 6   income             11572 non-null  object
 7   living_with        11572 non-null  object
 8   nation             11572 non-null  object
 9   phone              11572 non-null  object
 10  job                11572 non-null  object
 11  fb_freq            11572 non-null  int64 
 12  yt_freq            11572 non-null  int64 
 13  tik_freq           11572 non-null  int64 
 14  use_less_than_2GB  11572 non-null  int64 
 15  use_2GB_to_4GB     11572 non-null  int64 

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,tik_freq,use_less_than_2GB,use_2GB_to_4GB,marriage_status,number_child
0,11156,Rachel Gibbs,Female,21,Unknown,Unemployed,9.384000e+08,Unmarried_2,Australia,(08)-8012-7556,Astronomer,0,0,9,8,3,Unmarried,2
1,4297,Karen Anderson,Other,22,Unknown,Unemployed,9.840000e+08,Unmarried_2,England,943-646-5203,Air cabin crew,0,0,23,7,2,Unmarried,2
2,13301,김지원,Female,24,Unknown,Unemployed,1.063200e+09,Unmarried_1,Korea,010-4500-9888,Mechanical engineering technician and researcher,0,0,23,5,2,Unmarried,1
3,9920,Elisabeth Wähner,Female,24,Unknown,Unemployed,1.065600e+09,Unmarried_1,Denmark,+49(0) 587406963,Medical informatics specialist,0,0,21,7,2,Unmarried,1
4,8424,Sra. Maria Luiza Nogueira,Female,25,Unknown,Unemployed,1.100000e+09,Unmarried_1,Brazil,(084) 0568 1445,Ufologist,0,0,22,5,3,Unmarried,1


In [ ]:
# Chạy cho context
def run_context():
    p = DataPreprocessor()
    pipe = DatasetPipeline(p)

    pipe.process_context("context.csv", "clean_context.csv")
    print("✔ context.csv processed → clean_context.csv")

run_context()

# Đọc lại file clean_context
clean_context = pd.read_csv("clean_context.csv")
print("\n 5 dòng đầu tiên sau khi làm sạch context.csv:")
clean_context.head()

Đã đọc dữ liệu thành công!
Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11572 entries, 0 to 11571
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            11572 non-null  int64 
 1   purpose       11572 non-null  object
 2   go_with       11572 non-null  object
 3   weather       11572 non-null  object
 4   time          11572 non-null  object
 5   viettel_no_0  11572 non-null  int64 
 6   viettel_no_1  11572 non-null  int64 
 7   viettel_no_2  11572 non-null  int64 
 8   to_hanoi      11572 non-null  int64 
 9   to_other      11572 non-null  int64 
 10  score         11572 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 994.6+ KB
None

Số dòng và cột:
(11572, 11)

 5 dòng đầu tiên:
      id purpose       go_with    weather     time  viettel_no_0  \
0  11156  Travel      Al?!%one  &&!!Sunny    13:00             1   
1   4297  Travel  F!ri?end%(s)    Sun!~ny  9:00 AM        

,id,purpose,go_with,weather,time,viettel_no_0,viettel_no_1,viettel_no_2,to_hanoi,to_other,score
0,11156,Travel,Alone,Sunny,13:00,1,0,0,0,1,0
1,4297,Travel,Friends,Sunny,09:00,0,1,0,0,1,0
2,13301,Travel,Friends,Sunny,15:00,0,1,0,0,1,0
3,9920,Travel,Friends,Sunny,14:00,0,1,0,0,1,30
4,8424,Travel,Family,Sunny,10:00,0,1,0,0,1,0


In [ ]:
# Chạy cho mobile_plan_user
def run_mobile_plan_user():
    p = DataPreprocessor()
    pipe = DatasetPipeline(p)

    pipe.process_mobile_plan_user("mobile_plan_user.csv", "clean_mobile_plan_user.csv", "mobile_plan_user_agg.csv")
    print("✔ mobile_plan_user.csv processed → clean_mobile_plan_user.csv")

run_mobile_plan_user()

# Đọc file clean_mobile_plan_user.csv và mobile_plan_user_agg.csv
clean_mobile_plan_user = pd.read_csv("clean_mobile_plan_user.csv")
print("\n 5 dòng đầu tiên sau khi làm sạch mobile_plan_user.csv:")
print(clean_mobile_plan_user.head())
print("\n 5 dòng đầu tiên của bảng tổng hợp mobile_plan_user_agg.csv:")
mobile_plan_user_agg = pd.read_csv("mobile_plan_user_agg.csv")
print(mobile_plan_user_agg.head())

Đã đọc dữ liệu thành công!
Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45321 entries, 0 to 45320
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           45321 non-null  float64
 1   mobile_plan  42606 non-null  object 
 2   accept       42600 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.0+ MB
None

Số dòng và cột:
(45321, 3)

 5 dòng đầu tiên:
        id      mobile_plan  accept
0  11156.0       DATASILVER     1.0
1   4297.0  SOCIALMEDIAGOLD     1.0
2  13301.0       DATASILVER     1.0
3   9920.0  SOCIALMEDIAGOLD     1.0
4   8424.0       DATASILVER     1.0

Thống kê mô tả:
                 id        accept
count  45321.000000  42600.000000
mean   10533.209704      0.177887
std     5464.227694      0.382422
min     1000.000000      0.000000
25%     5803.000000      0.000000
50%    10557.000000      0.000000
75%    15268.000000      0.000000
max    19999.000000      1.

In [ ]:
# Chạy cho mobile_plan_attr
def run_mobile_plan_attr():
    p = DataPreprocessor()
    pipe = DatasetPipeline(p)

    pipe.process_mobile_plan_attr("mobile_plan_attr.csv", "clean_mobile_plan_attr.csv")
    print("✔ mobile_plan_attr.csv processed → clean_mobile_plan_attr.csv")

run_mobile_plan_attr()

# In ra bảng sạch sau khi xử lý mobile_plan_attr:
clean_mobile_plan_attr = pd.read_csv("clean_mobile_plan_attr.csv")
print("\n 5 dòng đầu tiên sau khi làm sạch mobile_plan_attr.csv:")
clean_mobile_plan_attr.head()


Đã đọc dữ liệu thành công!
Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   mobile_plan  5 non-null      object
 1   description  5 non-null      object
 2   price        5 non-null      int64 
 3   duration     5 non-null      object
dtypes: int64(1), object(3)
memory usage: 292.0+ bytes
None

Số dòng và cột:
(5, 4)

 5 dòng đầu tiên:
       mobile_plan                                        description   price  \
0       DATASILVER                               2GB/ day; high speed  100000   
1         DATAGOLD                               5GB/ day; high speed  200000   
2      SOCIALMEDIA  1 GB/day; Unlimited for Tik Tok, Facebook, You...  150000   
3  SOCIALMEDIAGOLD  3GB/ day; high speed; Unlimited for Tik Tok, F...  250000   
4         DATACALL  2GB/ day; high speed; 300 mins call for extern...  200000   

  duration  
0    

,mobile_plan,description,price,duration
0,DATASILVER,2GB/ day; high speed,100000,5d
1,DATAGOLD,5GB/ day; high speed,200000,5d
2,SOCIALMEDIA,"1 GB/day; Unlimited for Tik Tok, Facebook, You...",150000,3d
3,SOCIALMEDIAGOLD,"3GB/ day; high speed; Unlimited for Tik Tok, F...",250000,3d
4,DATACALL,2GB/ day; high speed; 300 mins call for extern...,200000,5d


In [ ]:
# Join 3 bảng dữ liệu sau khi tiền xử lý theo id: clean_user, clean_context, mobile_plan_user_agg
data_train_clean = pd.merge(clean_user, clean_context, on='id', how='left')
# Join với mobile_plan_user_agg
data_train_clean = pd.merge(data_train_clean, mobile_plan_user_agg, on='id', how='left')

data_train_clean

,id,name,gender,age,education,profession,income,living_with,nation,phone,...,viettel_no_1,viettel_no_2,to_hanoi,to_other,score,number_of_suggestions,number_of_plans,number_of_accept,number_of_accept_2,mobile_plan
0,11156,Rachel Gibbs,Female,21,Unknown,Unemployed,9.384000e+08,Unmarried_2,Australia,(08)-8012-7556,...,0,0,0,1,0,3,2,1.0,1,DATASILVER
1,4297,Karen Anderson,Other,22,Unknown,Unemployed,9.840000e+08,Unmarried_2,England,943-646-5203,...,1,0,0,1,0,4,3,1.0,1,SOCIALMEDIAGOLD
2,13301,김지원,Female,24,Unknown,Unemployed,1.063200e+09,Unmarried_1,Korea,010-4500-9888,...,1,0,0,1,0,5,3,1.0,1,DATASILVER
3,9920,Elisabeth Wähner,Female,24,Unknown,Unemployed,1.065600e+09,Unmarried_1,Denmark,+49(0) 587406963,...,1,0,0,1,30,3,2,1.0,1,SOCIALMEDIAGOLD
4,8424,Sra. Maria Luiza Nogueira,Female,25,Unknown,Unemployed,1.100000e+09,Unmarried_1,Brazil,(084) 0568 1445,...,1,0,0,1,0,2,2,1.0,1,DATASILVER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11567,11602,Austin Barber,Male,26,Bachelor,Sales & Related,1.968000e+09,Single_0,England,001-798-297-3345x825,...,1,0,0,1,0,5,4,1.0,1,SOCIALMEDIA
11568,5645,남상철,Male,26,Bachelor,Sales & Related,2.013600e+09,Single_0,Korea,043-900-0635,...,1,0,1,0,0,2,2,0.0,0,Unknown
11569,15477,John Blanchard,Male,30,Bachelor,Sales & Related,1.838400e+09,Single_0,Us,230-283-6789x89147,...,0,0,1,0,13,2,2,0.0,0,Unknown
11570,8885,Troy Green,Male,30,Bachelor,Sales & Related,2.087500e+09,Single_0,Australia,+61.472.119.694,...,0,1,0,1,0,2,2,0.0,0,Unknown


In [ ]:
# Xuất file sạch cuối cùng sau khi join để dùng cho EDA và train mô hình
data_train_clean.to_csv("data_train_cleaned.csv", index = False)

## **2. Áp dụng cho tập test**

In [ ]:
# Chạy cho user_test
def run_user_test():
    p = DataPreprocessor()
    pipe = DatasetPipeline(p)

    pipe.process_user("user_test.csv", "clean_user_test.csv")
    print("✔ user_test.csv processed → clean_user_test.csv")

run_user_test()

# Đọc lại file clean_user_test
clean_user_test = pd.read_csv("clean_user_test.csv")
print("\n 5 dòng đầu tiên sau khi làm sạch user_test.csv:")
clean_user_test.head()

Đã đọc dữ liệu thành công!
Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 1112 non-null   int64 
 1   name               1112 non-null   object
 2   gender             1112 non-null   object
 3   age                1112 non-null   int64 
 4   education          764 non-null    object
 5   profession         1112 non-null   object
 6   income             1112 non-null   object
 7   living_with        1112 non-null   object
 8   nation             1112 non-null   object
 9   phone              1112 non-null   object
 10  job                1112 non-null   object
 11  fb_freq            1112 non-null   int64 
 12  yt_freq            1112 non-null   int64 
 13  tik_freq           1112 non-null   int64 
 14  use_less_than_2GB  1112 non-null   int64 
 15  use_2GB_to_4GB     1112 non-null   int64 
d

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,tik_freq,use_less_than_2GB,use_2GB_to_4GB,marriage_status,number_child
0,7067,김은주,Female,33,Masters,Unemployed,1.607500e+09,Married_1,Korea,053-265-1191,Clay product production machine operator,0,1,3,0,1,Married,1
1,2114,김지연,Female,34,Masters,Unemployed,1.600000e+09,Married_1,Korea,051-368-9724,Other transportation equipment maintenance wor...,0,1,1,1,1,Married,1
2,6582,Sharon Rodriguez,Female,34,Masters,Unemployed,1.627200e+09,Married_3,Australia,-5039,Actor,0,1,3,1,0,Married,3
3,11379,Mariah Peixoto,Female,34,Masters,Unemployed,1.792500e+09,Married_4,Brazil,(071) 6038 8054,Occupational therapist,0,1,3,0,0,Married,4
4,3986,김서영,Female,34,Masters,Unemployed,1.657500e+09,Married_1,Korea,054-274-1759,Product production manager,0,1,1,0,1,Married,1


In [ ]:
# Chạy cho context_test
def run_context_test():
    p = DataPreprocessor()
    pipe = DatasetPipeline(p)

    pipe.process_context("context_test.csv", "clean_context_test.csv")
    print("✔ context_test.csv processed → clean_context_test.csv")

run_context_test()

# Đọc lại file clean_context
clean_context_test = pd.read_csv("clean_context_test.csv")
print("\n 5 dòng đầu tiên sau khi làm sạch context_test.csv:")
clean_context_test.head()

Đã đọc dữ liệu thành công!
Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1112 non-null   int64 
 1   purpose       1112 non-null   object
 2   go_with       1112 non-null   object
 3   weather       1112 non-null   object
 4   time          1112 non-null   object
 5   viettel_no_0  1112 non-null   int64 
 6   viettel_no_1  1112 non-null   int64 
 7   viettel_no_2  1112 non-null   int64 
 8   to_hanoi      1112 non-null   int64 
 9   to_other      1112 non-null   int64 
 10  score         1112 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 95.7+ KB
None

Số dòng và cột:
(1112, 11)

 5 dòng đầu tiên:
      id purpose    go_with   weather     time  viettel_no_0  viettel_no_1  \
0   7067  Travel   A!!l?one    Su?nny    14:00             1             0   
1   2114  Travel  Friend(s)   ~S&unny  1:0

,id,purpose,go_with,weather,time,viettel_no_0,viettel_no_1,viettel_no_2,to_hanoi,to_other,score
0,7067,Travel,Alone,Sunny,14:00,1,0,0,0,1,5
1,2114,Travel,Friends,Sunny,13:00,0,1,0,0,1,4
2,6582,Travel,Family,Sunny,09:00,0,1,0,0,1,0
3,11379,Travel,Family,Sunny,12:00,1,0,0,0,1,0
4,3986,Visit,Alone,Sunny,17:00,1,0,0,1,0,3


In [ ]:
# Chạy cho mobile_plan_user_test
def run_mobile_plan_user_test():
    p = DataPreprocessor()
    pipe = DatasetPipeline(p)

    pipe.process_mobile_plan_user("mobile_plan_user_test.csv", "clean_mobile_plan_user_test.csv", "mobile_plan_user_agg_test.csv")
    print("✔ mobile_plan_user_test.csv processed → clean_mobile_plan_user_test.csv")

run_mobile_plan_user_test()

# Đọc file clean_mobile_plan_user_test.csv và mobile_plan_user_agg_test.csv
clean_mobile_plan_user_test = pd.read_csv("clean_mobile_plan_user_test.csv")
print("\n 5 dòng đầu tiên sau khi làm sạch mobile_plan_user_test.csv:")
print(clean_mobile_plan_user_test.head())
print("\n 5 dòng đầu tiên của bảng tổng hợp mobile_plan_user_agg_test.csv:")
mobile_plan_user_agg_test = pd.read_csv("mobile_plan_user_agg_test.csv")
print(mobile_plan_user_agg_test.head())

Đã đọc dữ liệu thành công!
Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1397 entries, 0 to 1396
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           1397 non-null   int64  
 1   mobile_plan  1310 non-null   object 
 2   accept       1305 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 32.9+ KB
None

Số dòng và cột:
(1397, 3)

 5 dòng đầu tiên:
      id      mobile_plan  accept
0   7067       DATASILVER     1.0
1   2114  SOCIALMEDIAGOLD     1.0
2   6582       DATASILVER     1.0
3  11379       DATASILVER     0.0
4   3986         DATACALL     1.0

Thống kê mô tả:
                 id       accept
count   1397.000000  1305.000000
mean   10539.619900     0.420690
std     5501.593657     0.493859
min     1014.000000     0.000000
25%     5991.000000     0.000000
50%    10256.000000     0.000000
75%    15352.000000     1.000000
max    19995.000000     1.000000

 Tỷ l

In [ ]:
# Chạy cho mobile_plan_attr_test
def run_mobile_plan_attr_test():
    p = DataPreprocessor()
    pipe = DatasetPipeline(p)

    pipe.process_mobile_plan_attr("mobile_plan_attr_test.csv", "clean_mobile_plan_attr_test.csv")
    print("✔ mobile_plan_attr_test.csv processed → clean_mobile_plan_attr_test.csv")

run_mobile_plan_attr_test()

# In ra bảng sạch sau khi xử lý mobile_plan_attr_test:
clean_mobile_plan_attr_test = pd.read_csv("clean_mobile_plan_attr_test.csv")
print("\n 5 dòng đầu tiên sau khi làm sạch mobile_plan_attr_test.csv:")
clean_mobile_plan_attr_test.head()


Đã đọc dữ liệu thành công!
Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   mobile_plan  5 non-null      object
 1   description  5 non-null      object
 2   price        5 non-null      int64 
 3   duration     5 non-null      object
dtypes: int64(1), object(3)
memory usage: 292.0+ bytes
None

Số dòng và cột:
(5, 4)

 5 dòng đầu tiên:
       mobile_plan                                        description   price  \
0       DATASILVER                               2GB/ day; high speed  100000   
1         DATAGOLD                               5GB/ day; high speed  200000   
2      SOCIALMEDIA  1 GB/day; Unlimited for Tik Tok, Facebook, You...  150000   
3  SOCIALMEDIAGOLD  3GB/ day; high speed; Unlimited for Tik Tok, F...  250000   
4         DATACALL  2GB/ day; high speed; 300 mins call for extern...  200000   

  duration  
0    

,mobile_plan,description,price,duration
0,DATASILVER,2GB/ day; high speed,100000,5d
1,DATAGOLD,5GB/ day; high speed,200000,5d
2,SOCIALMEDIA,"1 GB/day; Unlimited for Tik Tok, Facebook, You...",150000,3d
3,SOCIALMEDIAGOLD,"3GB/ day; high speed; Unlimited for Tik Tok, F...",250000,3d
4,DATACALL,2GB/ day; high speed; 300 mins call for extern...,200000,5d


In [ ]:
# Join 3 bảng dữ liệu sau khi tiền xử lý theo id: clean_user_test, clean_context_test, mobile_plan_user_agg_test
data_test_clean = pd.merge(clean_user_test, clean_context_test, on='id', how='left')
# Join với mobile_plan_user_agg_test
data_test_clean = pd.merge(data_test_clean, mobile_plan_user_agg_test, on='id', how='left')

data_test_clean

,id,name,gender,age,education,profession,income,living_with,nation,phone,...,viettel_no_1,viettel_no_2,to_hanoi,to_other,score,number_of_suggestions,number_of_plans,number_of_accept,number_of_accept_2,mobile_plan
0,7067,김은주,Female,33,Masters,Unemployed,1.607500e+09,Married_1,Korea,053-265-1191,...,0,0,0,1,5,1,1,1.0,1,DATASILVER
1,2114,김지연,Female,34,Masters,Unemployed,1.600000e+09,Married_1,Korea,051-368-9724,...,1,0,0,1,4,2,1,1.0,1,SOCIALMEDIAGOLD
2,6582,Sharon Rodriguez,Female,34,Masters,Unemployed,1.627200e+09,Married_3,Australia,-5039,...,1,0,0,1,0,1,1,1.0,1,DATASILVER
3,11379,Mariah Peixoto,Female,34,Masters,Unemployed,1.792500e+09,Married_4,Brazil,(071) 6038 8054,...,0,0,0,1,0,1,1,0.0,0,Unknown
4,3986,김서영,Female,34,Masters,Unemployed,1.657500e+09,Married_1,Korea,054-274-1759,...,0,0,1,0,3,1,1,1.0,1,DATACALL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,19084,박서윤,Female,37,Bachelor,Education&Training&Library,1.047500e+09,Married_1,Korea,010-9131-7720,...,0,0,0,1,20,1,1,0.0,0,Unknown
1108,18221,Theodora Metz-Stolze,Female,40,Bachelor,Education&Training&Library,9.600000e+08,Married_2,Denmark,+49(0) 459976230,...,0,0,1,0,27,1,1,0.0,0,Unknown
1109,16530,Berta Fischer MBA.,Female,54,Bachelor,Unemployed,1.190000e+09,Married_4,Denmark,+49(0)1144 224313,...,1,0,0,1,0,1,1,0.0,0,Unknown
1110,1906,유서현,Female,72,Bachelor,Unemployed,9.475000e+08,Married_1,Korea,018-002-8075,...,1,0,0,1,7,1,1,0.0,0,Unknown


In [ ]:
# Xuất file sạch cuối cùng sau khi join để test mô hình
data_test_clean.to_csv("data_test_cleaned.csv", index = False)